In [6]:
   # create grid inside of canvas
    # assign ROI labels inside these grids

    
    # create canvas, add buttons as usual
    # divide canvas into 3 areas
    # add functionality for creation of points in other ROI's
    
    
    
   # """ Label Gui 1.0. Header/some functionality is from legacy code developed inside notebook 
    #found @ throneofshadow github."""
import tkinter as tk
import cv2
from tkinter import filedialog
from tkinter import messagebox
from tkinter.filedialog import askopenfilename
import random
import os
import math
from scipy.spatial import distance
from tkinter import filedialog
import numpy as np
import time
import pandas as pd
from PIL import Image, ImageTk
#from tkFileDialog import *
# global variables
dataDir = os.getcwd()    
filetype='.png'
armsize=40
wristsize=20
digitsize=10
pixwidth=1280/2
pixheight=1080/2
digitnum=2
arm=0
wrist=0
digit=0
scorer="Brett"
bodyparts="digit","arm","wrist"
class PointLabelGui(tk.Tk):
        def __init__(self, *args, **kwargs):
            global start # start variable for angular motion
            tk.Tk.__init__(self, *args, **kwargs)

            # initialize variables
            self.previous_x = self.previous_y = 0
            self.x = self.y = 0
            self.points_recorded = []
            self.filecontains=[]
            self.prev_file=[]
            self.grid()
            self.bttn_clicks = 0 
            self.arm=0
            self.wrist=0
            self.digit=0
            self.digit_tracker=0
            self.arm_tracker=0
            self.digit_tracker=0
            # declaring tk StringVars
            oc = tk.StringVar(self)
            oc.set("Select")
            folder_path = tk.StringVar()
            self.dataDir = tk.StringVar()
            self.dataDir.set(dataDir) 
            self.canvas1 = tk.Canvas(self, width=3*pixwidth, height=pixheight, bg="black",
                                        border=1, relief="sunken",cursor="cross")
            toolbar = tk.Frame(self)
            # BUTTONS
            clear_button = tk.Button(self, text="Clear Points", command=self.clear_all_points)
            clear_button.pack(in_=toolbar, side="left")
            btn_savepoints=tk.Button(self,text="Save Points",width=20,command=self.savepoints)
            btn_savepoints.pack(in_=toolbar,side="left")
            btn_deletepoints=tk.Button(self,text="Delete Points",width=20,command=self.delete_init)
            btn_deletepoints.pack(in_=toolbar,side="left")
            btn_label=tk.Button(self,text="Label Points",width=20,command=self.label_points)
            btn_label.pack(in_=toolbar,side="right")
            browse_btn=tk.Button(text="Browse Files",width=20, command=self.ddBrowseCallback)
            browse_btn.pack(in_=toolbar,side="left")
            prev_btn=tk.Button(text="Previous Image",width=20,command=self.prev_image)
            prev_btn.pack(in_=toolbar,side="right")
            btn_next=tk.Button(text="Next Image",width=20,command=self.next_image)
            btn_next.pack(in_=toolbar,side="right")
            btn_load=tk.Button(text="Load Image",width=20,command=self.image_return)
            btn_load.pack(in_=toolbar,side="left")
            btn_calib=tk.Button(text="Select Calibration Data",width=35, command=self.ddBrowseCallbackCalib)
            btn_calib.pack(in_=toolbar,side="left")
            o = tk.OptionMenu(self, oc, "arm","wrist","digit", command=self.segment_inputs)
            o.pack(in_=toolbar,side="right")
            toolbar.pack(side="top", fill="x")
            self.canvas1.pack(side="left", fill="both", expand=True)
            self.canvas1.bind("<Button-1>", self.find_point)
            self.canvas1.bind( "<B1-Motion>", self.tell_me_where_you_are )
            self.canvas1.bind("<Button-2>",self.delete_point)
            # initialize calibration objects
            self.create_P1_2()
            self.create_P1_3()
            self.create_P2_1()
            self.create_P2_3()
            self.create_P3_1()
            self.create_P3_2()


        #### Code Below is Exclusively for Image Selection/Browsing #####

    ################# WIP ##############
        def next_image(self):
            self.file_choice=random.choice(self.filecontains)
            file_path = (str(self.dataDir.get())+'/'+str(self.file_choice))
            image = cv2.imread(file_path)  
            b,g,r = cv2.split(image)
            img = cv2.merge((r,g,b))
            im = Image.fromarray(img)
            self.imgtk = ImageTk.PhotoImage(image=im) 
            self.canvas1.create_image(pixwidth,pixheight,image=self.imgtk,anchor='nw') 
            self.prev_file.append(self.file_choice)
            self.filecontains.remove(self.file_choice)
            
        def prev_image(self):
            self.file_choice=self.prev_file[-1]
            del self.prev_file[-1]
            file_path = (str(self.dataDir.get())+'/'+str(self.file_choice))
            print(file_path)
            image = cv2.imread(file_path)
            b,g,r = cv2.split(image)
            img = cv2.merge((r,g,b))
            im = Image.fromarray(img)
            self.imgtk = ImageTk.PhotoImage(image=im) 
            self.canvas1.create_image(pixwidth,pixheight,image=self.imgtk,anchor='nw') 
            
            
        def image_select(self):
            self.file_choice=random.choice(self.filecontains) # selects random file
            file_path=(str(self.dataDir.get())+'/'+str(self.file_choice))
            image = cv2.imread(file_path)
            b,g,r = cv2.split(image)
            img = cv2.merge((r,g,b))
            im = Image.fromarray(img)
            self.imgtk = ImageTk.PhotoImage(image=im) 
            self.canvas1.create_image(pixwidth,pixheight,image=self.imgtk,anchor='nw') 
            self.filecontains.remove(self.file_choice)# do initial deleting to clean array of image
            self.prev_file.append(self.file_choice)
            
        def image_return(self):
            for file in os.listdir(str(self.dataDir.get())+'/'):
                if file.endswith("png"):
                    self.filecontains.append(file)
                    print(self.filecontains)
            self.image_select()
            
        def ddBrowseCallback(self):
            self.dataDir.set(filedialog.askdirectory())
            
        def ddBrowseCallbackCalib(self):
            self.calib_filename = askopenfilename()
            self.calib_extrinsics= pd.read_csv(self.calib_filename)
            # set calibration functions here
            
        ##### Functions for choosing color, line, specifying what point we are labeling
        def choose_color(self):
            if self.arm==1:
                return 'red'
            if self.wrist==1:
                return 'yellow'
            if self.digit == 1:
                return 'blue'
            if self.digit >= 2:
                return 'green'
        def choose_tag(self):
            if self.arm==1:
                return "arm"
            if self.wrist==1:
                return "wrist"
            if self.digit == 1:
                return 'digit'

        def segment_inputs(self,input):
                if input == "arm":
                    self.arm=1
                    self.digit=0
                    self.wrist=0
                if input == "wrist":
                    self.arm=0
                    self.digit=0
                    self.wrist=1
                if input=="digit":
                    self.arm=0
                    self.digit+=1
                    self.wrist=0
### UNDER THE HOOD EXTRINSIC TRANSFORM WORK/CREATION ####
        def create_point_1_2(self):
            updated_coords=self.P1_2*self.main_coords # update coordinates into new pixelframe
            self.canvas1.create_line(updated_coords[0],updated_coords[1],updated_coords[0]+2,updated_coords[1]+2,
                                    fill=self.choose_color(),tag=self.choose_tag())
        def create_point_1_3(self):
            updated_coords=self.P1_3*self.main_coords # update coordinates into new pixelframe
            self.canvas1.create_line(updated_coords[0],updated_coords[1],updated_coords[0]+2,updated_coords[1]+2,
                                    fill=self.choose_color(),tag=self.choose_tag())
        def create_point_2_1(self):
            updated_coords=self.P2_1*self.main_coords # update coordinates into new pixelframe
            self.canvas1.create_line(updated_coords[0],updated_coords[1],updated_coords[0]+2,updated_coords[1]+2,
                                    fill=self.choose_color(),tag=self.choose_tag())
        def create_point_2_3(self):
            updated_coords=self.P2_3*self.main_coords # update coordinates into new pixelframe
            self.canvas1.create_line(updated_coords[0],updated_coords[1],updated_coords[0]+2,updated_coords[1]+2,
                                    fill=self.choose_color(),tag=self.choose_tag()) 
        def create_point_3_1(self):
            updated_coords=self.P3_1*self.main_coords # update coordinates into new pixelframe
            self.canvas1.create_line(updated_coords[0],updated_coords[1],updated_coords[0]+2,updated_coords[1]+2,
                                    fill=self.choose_color(),tag=self.choose_tag())
        def create_point_3_2(self):
            updated_coords=self.P3_2*self.main_coords # update coordinates into new pixelframe
            self.canvas1.create_line(updated_coords[0],updated_coords[1],updated_coords[0]+2,updated_coords[1]+2,
                                    fill=self.choose_color(),tag=self.choose_tag())
        #### Create Projection Vectors #####
        def create_P1_2(self):
            self.P1_2=1
        def create_P1_3(self):
            self.P1_3=1
        def create_P2_1(self):
            self.P2_1=1
        def create_P2_3(self):
            self.P2_3=1
        def create_P3_1(self):
            self.P3_1=1
        def create_P3_2(self):
            self.P3_2=1
    #### Find specific events ####
        def find_events(self,event):
                event_coords=np.array(event.x,event.y)
                events=self.canvas1.find_all()
                dst=[]
                for items in enumerate(self.events):
                    slowski=self.canvas1.coords(items)
                    event_coords=np.array(slowski[0],slowski[1])
                    dst.append(distance.euclidean(items,event_coords))
                index_min = min(xrange(len(dst)), key=values.__getitem__)
                self.selected_event_1=events(index_min) # obtains item ID for use in deletion etc
                ### Pull Coords from event, transform and delete the other points
                slowski=self.canvas1.coords(self.selected_event)
                if slowski[0] <= pixwidth:
                    #generate correct coords
                    xform_x1=[slowski[0]+pixwidth,slowski[1]]*P1_2
                    xform_x2=[slowski[0]+(2*pixwidth),slowski[1]]*P1_3
                    self.selected_event_2 = self.canvas1.find_closest(xform_x1[0],xform_x1[1])
                    self.selected_event_3 = self.canvas1.find_closest(xform_x2[0],xform_x2[1])
                    
                if slowski[0] <= pixwidth*2 and slowski[0] > pixwidth:
                    xform_x1=[slowski[0]-pixwidth,slowski[1]]*P2_1
                    xform_x2=[slowski[0]+(pixwidth),slowski[1]]*P2_3
                    self.selected_event_2 = self.canvas1.find_closest(xform_x1[0],xform_x1[1])
                    self.selected_event_3 = self.canvas1.find_closest(xform_x2[0],xform_x2[1])
                if slowski[0] <= pixwidth*3 and slowski[0] > pixwidth*2:
                    xform_x1=[slowski[0]-(2*pixwidth),slowski[1]]*P2_1
                    xform_x2=[slowski[0]-(pixwidth),slowski[1]]*P2_3
                    self.selected_event_2 = self.canvas1.find_closest(xform_x1[0],xform_x1[1])
                    self.selected_event_3 = self.canvas1.find_closest(xform_x2[0],xform_x2[1])     
        def find_point(self,event):
            self.mainx=event.x
            self.mainy=event.y 
            # check and see if we have exceeded how many points per segment we can have
            
            if self.mainx <= pixwidth:
                    self.main_coords=[self.mainx,self.mainy]
                    colorz=self.choose_color()
                    tag=self.choose_tag()
                    self.previous_event=self.canvas1.create_line(self.mainx+2, self.mainy+2, 
                                    self.mainx, self.mainy,fill=colorz,tag=tag)
                    self.create_point_1_2()
                    self.create_point_1_3()
            if self.mainx <= pixwidth*2 and self.mainx > pixwidth:
                    self.mainx = self.mainx -pixwidth
                    self.main_coords=[self.mainx,self.mainy]
                    colorz=self.choose_color()
                    tag=self.choose_tag()
                    self.previous_event=self.canvas1.create_line(self.mainx+2, self.mainy+2, 
                                    self.mainx, self.mainy,fill=colorz,tag=tag)
                    self.create_point_2_1()
                    self.create_point_2_3()
            if self.mainx <= pixwidth*3 and self.mainx > pixwidth*2:
                    self.mainx=self.mainx=(pixwidth*2)
                    self.main_coords=[self.mainx,self.mainy]
                    colorz=self.choose_color()
                    tag=self.choose_tag()
                    self.previous_event=self.canvas1.create_line(self.mainx+2, self.mainy+2, 
                                    self.mainx, self.mainy,fill=colorz,tag=tag)
                    self.create_point_3_1()
                    self.create_point_3_2()
        ### Code Below is for Saving and Appending Points Between Frames/Sessions ####

        def generate_header(self):
            self.scorer=scorer
            self.bodyparts=bodyparts
            self.scorer_array=["Scorer"]
            self.bodyparts_array=["Bodyparts"]
            self.coords_array=["coords"]
            for i in bodyparts:
                self.scorer_array.append(scorer)
                self.bodyparts_array.append(bodyparts[i])
                self.coords_array.append('x')
                self.coords_array.append('y')
                
        
        def frame_savepoints(self):
            # call into function to find labeled_coords item
            self.get_labeled_coords()
            for datafile in os.listdir(str(self.dataDir.get())):
                if self.prefix_stamp in datafile and datafile.endswith('.h5'):
                        self.dataframe=pd.read_hdf(datafile)    
                        self.dataframe.append(self.labeled_coords)
                else:# create from scratch
                    self.generate_header()
                    self.dataframe = pd.DataFrame()
                    self.dataframe.append([self.scorer_array])
                    self.dataframe.append([self.bodyparts_array])
                    self.dataframe.append([self.coord_array])


        def get_labeled_coords(self):
            pass
            # Find each object for a given tag
            

        def savepoints(self):
            pass

      ### Code Below is for Point Labeling w/ Line's Exclusively ####

        def update_count(self):
            self.bttn_clicks += 1

        def tell_me_where_you_are(self, event):
            self.previous_x = event.x
            self.previous_y = event.y
        def label_points(self):
            self.canvas1.bind("<Button-1>", self.find_point)
            self.canvas1.bind( "<B1-Motion>", self.tell_me_where_you_are )
            self.canvas1.bind("<Button-2>",self.delete_point)
       
        def add_line(self):
            self.canvas1.create_line(self.previous_x, self.previous_y, 
                                    self.x, self.y,fill="yellow")
                #self.canvas1.create_oval(self.previous_x,self.previous_y,self.x,self.y)
        def delete_init(self):
            self.canvas1.bind("<Button-1>", self.delete_point)
        def delete_point(self,event):
            self.find_events(event)
            self.canvas1.delete(self.selected_event_1)
            self.canvas1.delete(self.selected_event_2)
            self.canvas1.delete(self.selected_event_3)
        ### Code here is utilities/smaller functions ####     
        def clear_all_points(self):
            self.canvas1.delete("all") # delete all canvas objects
            self.bttn_clicks=0
        def onQuit(self):
            self.window.destroy()  

app = PointLabelGui()
app.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/kallanved/anaconda3/envs/deeplabcut/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-6-cd683307f4a6>", line 159, in image_return
    self.image_select()
  File "<ipython-input-6-cd683307f4a6>", line 139, in image_select
    self.file_choice=random.choice(self.filecontains) # selects random file
  File "/home/kallanved/anaconda3/envs/deeplabcut/lib/python3.6/random.py", line 260, in choice
    raise IndexError('Cannot choose from an empty sequence') from None
IndexError: Cannot choose from an empty sequence


['rat.png']
rat.png
/home/kallanved/tracking/ImageReconstruction/rat.png


Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/kallanved/anaconda3/envs/deeplabcut/lib/python3.6/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-6-cd683307f4a6>", line 109, in next_image
    self.file_choice=random.choice(self.filecontains)
  File "/home/kallanved/anaconda3/envs/deeplabcut/lib/python3.6/random.py", line 260, in choice
    raise IndexError('Cannot choose from an empty sequence') from None
IndexError: Cannot choose from an empty sequence


/home/kallanved/tracking/ImageReconstruction/rat.png
